In [ ]:
#!unzip face_mask-20200813T142403Z-001.zip
from google.colab import drive
drive.mount('/content/gdrive')
!pip install pyyaml==5.1 pycocotools>=2.0.1
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html


In [ ]:
                                                                                                     ###客製
%cd /content/gdrive/My Drive/test_Detectron2                                                     
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import numpy as np
import cv2
import random
import os
import matplotlib.pyplot as plt
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

#register_coco_instances("face_mask_train", {}, "./face_mask/train/train.json", "./face_mask/train")  ###客製
#register_coco_instances("face_mask_val", {}, "./face_mask/val/val.json", "./face_mask/val")          ###客製
cfg = get_cfg() 
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml") 
cfg.DATASETS.TRAIN = ("face_mask_train",)                                                            ###客製
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 4   
cfg.SOLVER.IMS_PER_BATCH = 1                                                                         ###客製
cfg.SOLVER.BASE_LR = 0.00025                                                                         ###客製
cfg.SOLVER.MAX_ITER = 400                                                                            ###客製
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1                                                                  ###客製
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)                                                           ###客製
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()
#%load_ext tensorboard
#%tensorboard --logdir output

In [ ]:
#from IPython.display import YouTubeVideo, display
#video = YouTubeVideo("8B6mXsj6jOY", width=500)
#display(video)
!pip install youtube-dl
!pip uninstall -y opencv-python-headless opencv-contrib-python
!apt install python3-opencv  # the one pre-installed have some issues
                                                                            ##客製   下載 youtube 的 video
!youtube-dl https://www.youtube.com/watch?v=8B6mXsj6jOY -f 22 -o video.mp4  
                                                                            ##客製    裁切長度=1分0秒
!ffmpeg -i video.mp4 -t 00:01:00 -c:v copy video-clip.mp4                   
!rm -r detectron2
                                                                            ##       下載github的detectron2
!git clone https://github.com/facebookresearch/detectron2                   

In [ ]:
thres = 0.6                                                         ##客製
                                                                    ##客製
%cd /content/gdrive/My Drive/test_Detectron2/detectron2/demo        
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import numpy as np
import cv2
import random
import os
import matplotlib.pyplot as plt
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import glob
import multiprocessing as mp
import time
import tqdm
from detectron2.data.detection_utils import read_image
from detectron2.utils.logger import setup_logger
from predictor import VisualizationDemo
WINDOW_NAME = "COCO detections"
                                                                      ##客製
%cd /content/gdrive/My Drive/test_Detectron2/                         
cfg = get_cfg() 
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")   ##客製
#cfg.DATASETS.TRAIN = ("face_mask_train",)
#cfg.DATASETS.TEST = ("face_mask_val", ) 
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = thres                         ##客製
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1                                   ##客製
cfg.freeze()

mp.set_start_method("spawn", force=True)
#setup_logger(name="fvcore")
#logger = setup_logger()
demo = VisualizationDemo(cfg)

video_input_path = '/content/gdrive/My Drive/test_Detectron2/video-clip.mp4'     ##客製
video = cv2.VideoCapture(video_input_path)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
frames_per_second = video.get(cv2.CAP_PROP_FPS)
num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
basename = os.path.basename(video_input_path)

output_path = '/content/gdrive/My Drive/test_Detectron2/video-clip_pred.mp4'     ##客製

if os.path.isdir(output_path):
    output_fname = os.path.join(output_path, basename)
    output_fname = os.path.splitext(output_fname)[0] + ".mkv"
else:
    output_fname = output_path
#assert not os.path.isfile(output_fname), output_fname
output_file = cv2.VideoWriter(
    filename=output_fname,
    # some installation of opencv may not support x264 (due to its license),
    # you can try other format (e.g. MPEG)
    fourcc=cv2.VideoWriter_fourcc(*"MPEG"),
    fps=float(frames_per_second),
    frameSize=(width, height),
    isColor=True,
    )
assert os.path.isfile(video_input_path)

for vis_frame in tqdm.tqdm(demo.run_on_video(video), total=num_frames):
    output_file.write(vis_frame)
    pass
video.release()
output_file.release()

/content/gdrive/My Drive/test_Detectron2/detectron2/demo
/content/gdrive/My Drive/test_Detectron2


100%|█████████▉| 1799/1800 [13:46<00:00,  2.18it/s]


In [ ]:
from detectron2.modeling import build_model
model = build_model(cfg)

from detectron2.checkpoint import DetectionCheckpointer
DetectionCheckpointer(model).load('./output/model_final.pth')  # load a file, usually from cfg.MODEL.WEIGHTS

#checkpointer = DetectionCheckpointer(model, save_dir="output")
#checkpointer.save("model_999")  # save to output/model_999.pth

[08/13 16:48:32 fvcore.common.checkpoint]: Loading checkpoint from ./output/model_final.pth
[08/13 16:48:33 fvcore.common.checkpoint]: Saving checkpoint to output/model_999.pth
